In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore', module="h5py")

In [ ]:
import pickle
import numpy as np
import itertools
from types import SimpleNamespace
import copy
from mpnetcdf4.utils import Timer
from mpnetcdf4.benchmarking import (
    with_stats,
    run_benchmark,
    dump_as_pickle,
    plot_benchmark_results)

from mpnetcdf4 import ReaderFactory
from tests import TEST_HDF_FILE, TEST_HDF_STACKED_FILE

display(TEST_HDF_STACKED_FILE)
!hostname && (hash free 2> /dev/null && free -m)

In [ ]:
def warmup_proc_pool(mp_factory):
    _, stats = run_benchmark(params, mp_factory,
                             chunk_scale=None,
                             src_roi=np.s_[:1,:,:])
    return stats

def prep_storage(pp, mp_factory):
    with mp_factory.open(pp.fname, 1) as f:
        return f.allocate(pp.measurements, pp.src_roi)

In [ ]:
quick_test = False
params = SimpleNamespace(
    fname=TEST_HDF_STACKED_FILE,
    mb=100,
    nprocs=15,
    src_roi=None,
    chunk_scale=(1,1,8),
    measurements=['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'coastal_aerosol'])

In [ ]:
if quick_test:
    params.src_roi = np.s_[:10, :400, :]
    params.nprocs = 4

In [ ]:
with Timer(message='Prepare'):
    mp_factory = ReaderFactory(params.nprocs, mb=params.mb)

warmup_proc_pool(mp_factory)

In [ ]:
out = prep_storage(params, mp_factory)
all_stats = []

for nprocs in range(1,params.nprocs+1):
    out, stats = run_benchmark(params, mp_factory, dst=out, nprocs=nprocs)
    all_stats.append(stats)

In [ ]:
dump_as_pickle(all_stats, 'benchmark-result-%02d.pickle')

In [ ]:
plot_benchmark_results(all_stats, fig=plt.figure(figsize=(16,4)))

In [ ]:
[(s.params.nprocs, s.t_total, 100*(s.t_total - s.elapsed)/s.t_total, s.throughput_mb)
 for s in all_stats]